Note: this example requires data from the LifeLines-DEEP project; you have to request access from them. (https://pubmed.ncbi.nlm.nih.gov/26319774/)

In [1]:
from TeraLasso import TeraLasso
from EiGLasso import EiGLasso
from GmGM import GmGM, Dataset
from GmGM.synthetic.generate_data import threshold_dictionary

import matplotlib.pyplot as plt
import numpy as np
import timeit

from cycler import cycler
linestyle_cycler = cycler('linestyle',['-','--',':','-.'])
color_cycler = cycler('color',['k','orange','m','darkgreen'])
plt.rc('axes', prop_cycle=linestyle_cycler + color_cycler)

import igraph as ig
import pandas as pd

import muon as mu
import anndata as ad
import scanpy as sc
from anndata import AnnData
import sklearn.cluster as clust

from typing import Literal, Union

/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/baileyandrew/mamb

# Setup

## Helper Functions

In [2]:
def calculate_assortativity(
    mudata: Union[mu.MuData, ad.AnnData],
) -> dict[Literal["Phylum", "Class", "Order", "Family", "Genus"], float]:
    global taxmat
    if isinstance(mudata, mu.MuData):
        precmat = mudata["metagenomics_shotgun"].varp[
            "metagenomics_shotgun-var_gmgm_connectivities"
        ].toarray()
    elif isinstance(mudata, ad.AnnData):
        precmat = mudata.varp[
            "var_gmgm_connectivities"
        ].toarray()
    else:
        raise ValueError("mudata must be of type mu.MuData or anndata.AnnData")
    g = ig.Graph.Adjacency(
        precmat != 0,
        mode="undirected",
        loops=False
    )
    return {
        tax_level: g.assortativity(taxmat[tax_level].cat.codes-1)
        for tax_level in ["Phylum", "Class", "Order", "Family", "Genus"]
    }

## Load Data

In [3]:
# Get the taxa
taxmat = pd.read_csv("../data/LL-Deep Data - Processed/ll-deep-taxmat.csv", index_col=0)
taxmat.columns = ["Domain", "Phylum", "Class", "Order", "Family", "Genus"]
taxmat = taxmat.fillna("Unknown")
taxmat = taxmat.apply(lambda x: x.astype("category"))

In [4]:
# Read in the mapping linking person ids from metabolomics and metagenomics
# (we actually constructed the latter two datasets such that the nth row matches
# up in all of them - but this still contains gender info)
map_df = pd.read_csv(
    "../data/LL-Deep Data - Processed/Map.csv",
    index_col=0
)
print(map_df.shape)

# Read in the metabolomics data
metabolomics_df = pd.read_csv(
    "../data/LL-Deep Data - Processed/Metabolomics.csv",
    index_col=0
)
print(metabolomics_df.shape)

# Read in the metagenomics data
metagenomics_df = pd.read_csv(
    "../data/LL-Deep Data - Processed/MetagenomicsShotgun.csv",
    index_col=0
)
old_shape = metagenomics_df.shape

# Only keep the species who appear in more than 20% of the people
#keep_idxs = ((counts > 0).sum(axis=0) > 0.2 * counts.shape[0]).values
#counts = counts.loc[:, keep_idxs]
#taxmat = taxmat.loc[keep_idxs, :]
keep_idxs = (metagenomics_df > 0).sum(axis=0) > 0.2 * metagenomics_df.shape[0]
metagenomics_df = metagenomics_df.loc[:, keep_idxs]
taxmat = taxmat.loc[keep_idxs.values, :]

print(old_shape, '->', metagenomics_df.shape)

(1054, 3)
(1054, 1183)
(1054, 3957) -> (1054, 564)


In [5]:
# Load our data into a MuData object
metabol_ann = AnnData(
    X = metabolomics_df.to_numpy()
)
metabol_ann.obs_names = metabolomics_df.index
metabol_ann.var_names = metabolomics_df.columns

metagen_ann = AnnData(
    X = metagenomics_df.to_numpy()
)
metagen_ann.obs_names = metabolomics_df.index # note this is the same as metabolomics
metagen_ann.var_names = metagenomics_df.columns

mudata = mu.MuData({
    "metabolomics": metabol_ann,
    "metagenomics_shotgun": metagen_ann
})

mudata.obs["Gender"] = map_df["Gender"].to_numpy()

# Log transform the data
sc.pp.log1p(mudata["metabolomics"])
sc.pp.log1p(mudata["metagenomics_shotgun"])

# GmGM

## w/o nonpara

In [6]:
GmGM(
    mudata,
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    random_state=0,
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='metagenomics_shotgun-var'
Calculating eigenvectors for axis='metabolomics-var'
Calculating eigenvalues...
@0: -1834610.2475790898 (-1834616.8682274285 + 6.620648338769167 + 0) ∆inf
Converged! (@14: -1955700.7343906139)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [7]:
calculate_assortativity(mudata)

{'Phylum': -0.0340116012287351,
 'Class': -0.07758431316582456,
 'Order': -0.10237227875338675,
 'Family': 0.01903746743427355,
 'Genus': -0.0758265373739494}

In [8]:
def to_time() -> None:
    GmGM(
        mudata,
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.7510346333999998

## w/ nonpara

In [9]:
GmGM(
    mudata,
    verbose=True,
    use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='metagenomics_shotgun-var'
Calculating eigenvectors for axis='metabolomics-var'
Calculating eigenvalues...
@0: -1835146.444140518 (-1835152.198621132 + 5.754480613893504 + 0) ∆inf
Converged! (@14: -1983139.8097202538)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [10]:
calculate_assortativity(mudata)

{'Phylum': -0.01083183431730019,
 'Class': 0.0038790432750807546,
 'Order': 0.012346823692702196,
 'Family': 0.010306609993891774,
 'Genus': -0.01737953363222772}

In [11]:
def to_time() -> None:
    GmGM(
        mudata,
        use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.7762516541999993

## w/o nonpara, 50pc

In [12]:
GmGM(
    mudata,
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating left eigenvectors of concatenated matricizations...
Calculating eigenvalues...
@0: 15499134.895770371 (-3379.8941838578976 + 15502514.789954228 + 0) ∆inf
Converged! (@16: 11421866.736312293)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [13]:
calculate_assortativity(mudata)

{'Phylum': 0.2161060638215047,
 'Class': 0.21280122938368426,
 'Order': 0.013449012768118976,
 'Family': -0.0006373060574091327,
 'Genus': 0.04490125109929295}

In [14]:
def to_time() -> None:
    GmGM(
        mudata,
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.3051413873999998

## w/ nonpara, 50pc

In [15]:
GmGM(
    mudata,
    verbose=True,
    use_nonparanormal_skeptic=True,
    nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating left eigenvectors of concatenated matricizations...
Calculating eigenvalues...
@0: 2299430.975742045 (-3457.1901852916317 + 2302888.1659273366 + 0) ∆inf
Converged! (@25: 1189653.7319726325)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [16]:
calculate_assortativity(mudata)

{'Phylum': 0.15367704079507166,
 'Class': 0.16089513681604936,
 'Order': -0.009558243935596528,
 'Family': 0.006441657993733161,
 'Genus': 0.03980574931696559}

In [17]:
def to_time() -> None:
    GmGM(
        mudata,
        use_nonparanormal_skeptic=True,
        nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

4.016988616599998

## w/o nonpara w/o metabol

In [18]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='var'
Calculating eigenvalues...
@0: -566088.016171067 (-566094.2859140849 + 6.269743017843555 + 0) ∆inf
Converged! (@10: -589782.8896450901)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [19]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': -0.03387022271612623,
 'Class': -0.07701084827385335,
 'Order': -0.10017418814769018,
 'Family': 0.01848634723723933,
 'Genus': -0.07653293336440652}

In [20]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.2553461332999959

## w/ nonpara, w/o metabol

In [21]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='var'
Calculating eigenvalues...
@0: -566118.05331034 (-566124.2453072045 + 6.191996864548216 + 0) ∆inf
Converged! (@11: -590597.1354492947)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [22]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': -0.01083183431730019,
 'Class': 0.0038790432750807546,
 'Order': 0.012346823692702196,
 'Family': 0.010306609993891774,
 'Genus': -0.01737953363222772}

In [23]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.2696349791000003

## w/o nonpara, w/o metabol, 50pc

In [24]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating SVD...
Calculating eigenvalues...
@0: 13946002.146203084 (-1686.312356038847 + 13947688.458559122 + 0) ∆inf
Converged! (@16: 8755450.68054035)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [25]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': 0.2118695284798763,
 'Class': 0.2082903370213568,
 'Order': 0.005898317258947495,
 'Family': -0.0072243868296697844,
 'Genus': 0.03619080034129565}

In [26]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.09322528740000138

## w/ nonpara, w/o metabol, 50pc

In [27]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    use_nonparanormal_skeptic=True,
    nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0
)

Centering...
Calculating eigenvectors...
	by calculating left eigenvectors of concatenated matricizations...
Calculating eigenvalues...
@0: 539020.7678061449 (-1718.0169078912443 + 540738.7847140362 + 0) ∆inf
Converged! (@19: 342570.79736465815)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [28]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': 0.16014617484457577,
 'Class': 0.17047380597786582,
 'Order': -0.0077096433385532735,
 'Family': 0.009502401204151589,
 'Genus': 0.030636717759824624}

In [29]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        use_nonparanormal_skeptic=True,
        nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "var": 1200 / 564**2,
            "obs": 1054 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

1.377092974800003

# TeraLasso

## w/o nonpara w/o metabol

In [30]:
ds = Dataset.from_AnnData(mudata["metagenomics_shotgun"])
# Changing this to 94.235 yields 4074, a big difference!
# But 94.24 yields 2774 and 94.23 yields 3014 so it is very nonmonotonic
# And 10 takes 70 minutes!!
TeraLasso(
    ds,
    94.23
    #use_nonparanormal_skeptic=True,
)
test = ds.to_AnnData()
test

/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/dataset.py:656: UserWarning: `use_highly_variable` was set to True but no highly variable indices were provided for axis 'obs'.  Thus we will use all indices for this axis.
  warnings.warn(
/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/dataset.py:656: UserWarning: `use_highly_variable` was set to True but no highly variable indices were provided for axis 'var'.  Thus we will use all indices for this axis.
  warnings.warn(


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [31]:
# want 1200 * 2 + 564 = 2964
564 * 564

318096

In [32]:
test.varp["var_gmgm_connectivities"]

<564x564 sparse array of type '<class 'numpy.float64'>'
	with 3178 stored elements in Compressed Sparse Row format>

In [33]:
calculate_assortativity(test)

{'Phylum': -0.04495282111959147,
 'Class': -0.03472514308788115,
 'Order': 0.025903799449613903,
 'Family': 0.016677534158937593,
 'Genus': -0.009260247972319766}

In [35]:
def to_time() -> None:
    ds = Dataset.from_AnnData(mudata["metagenomics_shotgun"])
    # Changing this to 94.235 yields 4074, a big difference!
    # But 94.24 yields 2774 and 94.23 yields 3014 so it is very nonmonotonic
    # And 10 takes 70 minutes!!
    TeraLasso(
        ds,
        94.23
        #use_nonparanormal_skeptic=True,
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

34.842702662499995

## w/ nonpara w/o metabol

In [ ]:
ds = Dataset.from_AnnData(mudata["metagenomics_shotgun"])
# Seems to produce no result for 2e-8, full result for 1.75e-8
# 1.9e-8 gives 572, 1.8e-9 gives 576
TeraLasso(
    ds,
    2e-8,
    use_nonparanormal_skeptic=True,
)
test = ds.to_AnnData()
test

/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/GmGM/dataset.py:656: UserWarning: `use_highly_variable` was set to True but no highly variable indices were provided for axis 'obs'.  Thus we will use all indices for this axis.
  warnings.warn(
/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/GmGM/dataset.py:656: UserWarning: `use_highly_variable` was set to True but no highly variable indices were provided for axis 'var'.  Thus we will use all indices for this axis.
  warnings.warn(


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [ ]:
test.varp["var_gmgm_connectivities"]

<564x564 sparse array of type '<class 'numpy.float64'>'
	with 564 stored elements in Compressed Sparse Row format>

In [ ]:
calculate_assortativity(test)

{'Phylum': nan, 'Class': nan, 'Order': nan, 'Family': nan, 'Genus': nan}

# GmGM + Wishart Prior

Seems to not do as well as the 50pc ones

In [87]:
# Crate prior connecting species in the same phylum
base = np.array(taxmat["Phylum"].cat.codes)[np.newaxis]
prior = (base == base.T).astype(float)
np.fill_diagonal(prior, 1.1)

In [37]:
prior.shape

(564, 564)

In [38]:
from GmGM.extras.prior import Wishart
Wishart(degrees_of_freedom=565, scale_matrix=prior)

In [41]:
GmGM(
    mudata,
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 1200 / 564**2,
        "obs": 1054 / 1054**2
    },
    threshold_method="overall",
    random_state=0,
    prior={"metagenomics_shotgun-var": Wishart(degrees_of_freedom=565, scale_matrix=prior)},
    force_posdef=False,
    tol=1e-8,
    max_iter=10_000,
    check_overstep_each_iter=True
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='metagenomics_shotgun-var'
Calculating eigenvectors for axis='metabolomics-var'
Calculating eigenvalues...
@0: -2012186.3188758958 (-2012196.156618618 + 9.837742722098143 + 0) ∆inf
@100: -4000111.173661599 (-4000140.4846803457 + 29.31101874671051 + 0) ∆0.01015944440468236
@200: -4416440.288689271 (-4416472.143440389 + 31.854751118273022 + 0) ∆5.557057901671376e-06
@300: -5117856.38038931 (-5118092.576344956 + 236.19595564541004 + 0) ∆0.0023056792894915693
@400: -5812650.093667895 (-5812727.400168344 + 77.30650045019614 + 0) ∆9.047376396690277e-05
@500: -5923729.293485547 (-5923798.099478133 + 68.80599258559872 + 0) ∆7.289930480091812e-07
@600: -5980733.627254723 (-5980803.93550007 + 70.30824534782006 + 0) ∆7.143961228600417e-05
@700: -6021514.194514115 (-6021585.576776063 + 71.38226194808793 + 0) ∆6.52734119159628e

/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/core/presparse_methods.py:207: UserWarning: Found negative eigenvalues in axis metabolomics-var
Setting them to 0
  warnings.warn(


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
      obsp:	'obs_gmgm_connectivities'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [42]:
calculate_assortativity(mudata)

{'Phylum': 0.1627053922612525,
 'Class': 0.10758731814697549,
 'Order': 0.01708691206799438,
 'Family': -0.0056812660115850705,
 'Genus': -0.003569931804283743}

In [105]:
GmGM(
    mudata,
    verbose=True,
    # use_nonparanormal_skeptic=True,
    # nonparanormal_evec_backend="COCA",
    # n_comps=50,
    to_keep={
        "metabolomics-var": 0,
        "metagenomics_shotgun-var": 3,
        #"metagenomics_shotgun-var": 1200 / 564**2,
        "obs": 0
    },
    #threshold_method="overall",
    threshold_method="rowwise",
    random_state=0,
    prior={"metagenomics_shotgun-var": Wishart(degrees_of_freedom=565, scale_matrix=np.linalg.inv(prior))},
    force_posdef=False,
    check_overstep_each_iter=True,
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='metagenomics_shotgun-var'
Calculating eigenvectors for axis='metabolomics-var'
Calculating eigenvalues...
@0: -2012156.0103766643 (-2012166.1222534997 + 10.111876835391195 + 0) ∆inf
Converged! (@18: -2597254.786183384)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
      obsp:	'obs_gmgm_connectivities'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [106]:
calculate_assortativity(mudata)

{'Phylum': 0.4712674151356942,
 'Class': 0.3061729680472627,
 'Order': 0.17896689454527723,
 'Family': 0.04729381448986717,
 'Genus': 0.003348427226214601}